In [20]:
import numpy as np
from IPython.display import HTML 
HTML('../style/course.css')
HTML('../style/code_toggle.html')
from mirpy import miriad

# Continuum subtraction

This chapter describes the methods that can be used to subtract radio continuum emission from single-dish data (?), visibility datasets or datacubes, resulting in data that contain spectral-line emission or absorption only. Here we will adopt the following notation:
- $S(l,m,\nu)$ is the sky brightness as a function of position (relative to a reference position $l_0,m_0$) and frequency; it is composed of a continuum and a spectral line term, $S(l,m,\nu) = S_\mathrm{c}(l,m,\nu) + S_\mathrm{s}(l,m,\nu)$; we assume $(l_0,m_0)$ to be both the pointing and phase-tracking centre;
- $A(l,m,\nu)$ is the primary beam pattern;
- $I(l,m,\nu) = S(l,m,\nu) \cdot A(l,m,\nu)$ is the apparent sky brightness;
- $B(l,m,\nu)$ is the point spread function;
- $I^\mathrm{D}(l,m,\nu)$ is the dirty cube obtained by convolving $I(l,m,\nu)$ with $B(l,m,\nu)$;
- $V(u,v,\nu)$ is the complex visibility, also composed of a continuum and a spectral line term $V(u,v,\nu) = V_\mathrm{c}(u,v,\nu) + V_\mathrm{s}(u,v,\nu)$;

## Outline

<ol start="1">
<li>[Somename](../chapter_01_somename/01_00_introduction.ipynb)</li>
    <ol start="1">
        <li>[Somename2](../chapter_01_somename/01_01_somename2.ipynb#somename:sec:somename2)</li>
            <ol start="1">
                <li>[Somename21](../chapter_01_somename/01_01_somename2.ipynb#somename:sec:somename21)</li>
                <li>[Somename22](../chapter_01_somename/01_01_somename2.ipynb#somename:sec:somename22)</li>
            </ol>
        <li>[Somename3](../chapter_01_somename/01_02_somename3.ipynb#somename:sec:somename3)</li>
            <ol start="1">
                <li>[Somename31](../chapter_01_somename/01_02_somename3.ipynb#somename:sec:somename31)</li>
                <li>[Somename32](../chapter_01_somename/01_02_somename3.ipynb#somename:sec:somename32)</li>
            </ol>
        <li>[1. Somename: References and further reading](../chapter_01_somename/01_references_and_further_reading.ipynb#somename:sec:references)</li>
            <ol start="1">
                <li>[1. Somename: References](../chapter_01_somename/01_02_somename3.ipynb#somename:sec:references1')</li>
                <li>[1. Somename: Further reading](../chapter_01_somename/01_02_somename3.ipynb#somename:sec:references2)</li>
            </ol>
        <li>[1. Somename: Appendix](../chapter_01_somename/01_appendix.ipynb#somename:sec:appendix')</li>
            <ol start="1">
                <li>[1. Somename: Appendix: Bla](../chapter_01_somename/01_02_somename3.ipynb#somename:sec:appendix1')</li>
            </ol>
    </ol>
</ol>

## 1. Overview of continuum subtraction methods

Continuum emission can be removed from interferometric data in a variety of ways, which can be grouped under the following 3 categories:
- estimating and subtracting the continuum emission from a cube $I(l,m,\nu)$
- estimating and subtracting the continuum emission from visibility spectra $V(u,v,\nu)$
- modelling the continuum emission $I_\mathrm{c}(l,m,\nu)$ from a continuum (possibly multi-frequency) image, and subtracting the resulting model from the visibilities $V(u,v,\nu)$

Each method has advantages and disadgantages, and often it is advisiable to use more than one of them. As usual, simpler methods are faster but less accurate. Important factors affecting their performance include:
- the fractional bandwidth over which one needs to subtract the continuum
- the distance of the continuum sources from the phase centre
- the brightness of the spectral line relative to the continuum

Below we describe these different methods, their advantages and disadvantages, and how the above factors come into play.

## 2. Continuum subtraction from a cube

If no continuum has been subtracted from the visibility dataset $V(u,v,\nu)$, each sightline ($l,m$) of the dirty cube $I^\mathrm{D}(l,m,\nu)$ will in principle include some level of continuum emission $I^\mathrm{D}_\mathrm{c}(l,m,\nu)\ne0$. This includes both emission at the position of real continuum sources and emission corresponding to their sidelobes. The sidelobes contribution could be removed by cleaning the cube. However, this is extremely time-consuming as it involves cleaning a large number of sources for each of the (usually many) spectral-line channels. Here we assume that this cannot be done and that the cube is dirty.

The basic idea of this method is to estimate and remove $I^\mathrm{D}_\mathrm{c}(l,m,\nu)$ along each sightline ($l,m$) independently. This can be done by fitting and subtracting a low order polynomial $I^\mathrm{D}_\mathrm{c,model}(l,m,\nu)=\sum_{n=0}^{N} a_n(l,m)\ \nu^n$ for each sightline. In general, for smaller fractional bandwidths the variation of $I(l,m,\nu)$ with frequency is more limited and, therefore, N can be smaller. The limiting case is one where it is sufficient to take the average of all channels (or a $0^\mathrm{th}$-order polynomial fit) as an estimate of a flat (i.e., frequency independent) continuum.

Note that the variation of $I^\mathrm{D}_\mathrm{c}(l,m,\nu)$ with frequency is determined by the intrinsic spectrum of the continuum sources contributing at the position $(l,m)$ combined with the variation of the primary beam $A(l,m,\nu)$ and the point-spread-function $B(l,m,\nu)$ with frequency.

There are a few aspects to consider.

- Channels containing spectral-line emission or absorption (or RFI) should be excluded before estimating the continuum. In principle this could be done independently for each sightline, allowing for spectral line sources with different observed frequency at different positions. In practice, current implementations in, e.g., CASA and MIRIAD, allow users to only indicate what channels should be excluded for the cube as a whole. This is acceptable when working on a dirty cube, since the PSF spreads line emission and absorption at a given $(l,m)$ to all positions at that channel. A significant improvement over current implementations, and one that would allievate some of these issues, would be to include simple outlier rejection algorithms which would effectively allow for a position-dependent channel selection.
- Identifying which channels should be excluded from the contimuum estimate is not always straightforward. In particular, spectral-line emission or absorption much fainter than the continuum sources in the field may not be easy to indentify in a dirty cube, where the sidelobes of the continuum sources occupy much of the cube for all channels. Therfore, cleaning may be required and a few iterations of cleaning and continuum subtraction may be needed before  the line-free channels are identified reliably.
- For a dirty cube, the variation of $I_\mathrm{c}(l,m,\nu)$ with frequency is in general more complex along sightlines dominated by sidelobes than at the position of an actual continuum source. Clearly this issue is more important for larger fractional bandwidths. Cubes made with MIRIAD have a frequency independent PSF because they change the pixel scale with frequency. In this case, the issue is that real sources move across pixels as a function of frequency, and again the frequency behaviour for a given sightline is complex.

This method is robust against calibration errors.

## 3. Continuum subtraction from the visibilities

UVLIN approach. Advantage that it is independent of g(t) calibration. Limitation of distance from phase centre.

## 4. Continuum subtraction from an image-based model

Modelling with MFS, then subtraction. Limited by accuracy of calibration and of continuum model.

## 5. Combining the above approaches

E.g., subtract a model, especially for distant sources, then UVLIN.

### Bibliography

<a href="http://adsabs.harvard.edu/abs/1992A%26A...258..583C">Cornwell, Uson & Addad 1992, A&A, 258, 583</a>

<a href="http://adsabs.harvard.edu/abs/1999ASPC..180..229R">Rupen 1999, ASPC, 180, 229</a>

<a href="http://adsabs.harvard.edu/abs/1994A%26AS..107...55S">Salt 1994, A&AS, 107, 55</a>

<a href="http://adsabs.harvard.edu/abs/1983ApJ...267..528V">van Gorkom & Ekers 1983, ApJ, 267, 528</a>

<a href="http://adsabs.harvard.edu/abs/1989ASPC....6..341V">van Gorkom & Ekers 1989, ASPC, 6, 341</a>

<a href="http://adsabs.harvard.edu/abs/1990A%26A...239L...5V">van Langevelde & Cotton 1990, A&A, 239L, 5

<a href="http://adsabs.harvard.edu/abs/2015MNRAS.453.2399W">Wang et al. 2015, MNRAS, 453, 2399</a>

### Chapter Editors
* Name 1
* Name 2

#### Chapter Contributors

* Name 3 (**1.0**, **1.1**)
* Name 4 (**1.1**)

***

   * Next: [1.1 Somename 2](01_01_somename2.ipynb)
